based on https://en.wikipedia.org/wiki/Lambda_calculus#Encoding_datatypes

## Helpers

In [1]:
def _increment(x):
    return x + 1

def to_int(church_numeral):
    return church_numeral(_increment)(0)

def check_numeral(church_numeral, numeral):
    assert to_int(church_numeral) == numeral

import inspect

def get_definition(f):
    return inspect.getsource(f).split("=")[1].strip()


    

## Arithmetics

Numerals:

*   0: λf.λx.x
*   1: λf.λx.f x
*   2: λf.λx.f (f x)
*   3: λf.λx.f (f (f x))

In [2]:
_ZERO = lambda f: lambda x: x
_ONE = lambda f: lambda x: f(x)
_TWO = lambda f: lambda x: f(f(x))
_THREE = lambda f: lambda x: f(f(f(x)))

check_numeral(_ZERO, 0)
check_numeral(_ONE, 1)
check_numeral(_TWO, 2)
check_numeral(_THREE, 3)

SUCC := λn.λf.λx.f (n f x)

In [3]:
_SUCC = lambda n: lambda f: lambda x: f(n(f)(x))

check_numeral(_SUCC(_ZERO), 1)
check_numeral(_SUCC(_SUCC(_ZERO)), 2)
check_numeral(_SUCC(_SUCC(_SUCC(_ZERO))), 3)

PLUS := λm.λn.λf.λx.m f (n f x)

In [4]:
_PLUS = lambda m: lambda n: lambda f: lambda x: m(f)(n(f)(x))

_FIVE = _SUCC(_SUCC(lambda f: lambda x: f(f(f(x)))))

check_numeral(_PLUS(_THREE)(_ZERO), 3)
check_numeral(_PLUS(_THREE)(_ONE), 4)
check_numeral(_PLUS(_THREE)(_FIVE), 8)

MULT := λm.λn.λf.m (n f)

In [5]:
_MULT = lambda m: lambda n: lambda f: m(n(f))

check_numeral(_MULT(_THREE)(_ZERO), 0)
check_numeral(_MULT(_THREE)(_ONE), 3)
check_numeral(_MULT(_THREE)(_FIVE), 15)

POW := λb.λe.e b

In [6]:
_POWER = lambda m: lambda n: n(m)

check_numeral(_POWER(_THREE)(_ZERO), 1)
check_numeral(_POWER(_THREE)(_ONE), 3)
check_numeral(_POWER(_THREE)(_FIVE), 243)

PRED := λn.λf.λx.n (λg.λh.h (g f)) (λu.x) (λu.u)

In [7]:
_PRED = lambda n: lambda f: lambda x: n(lambda g: lambda h: h(g(f)))(lambda u: x)(lambda u: u)

check_numeral(_PRED(_ONE), 0)
check_numeral(_PRED(_TWO), 1)
check_numeral(_PRED(_THREE), 2)

SUB := λm.λn.n PRED m

In [8]:
_SUB = lambda m: lambda n: n(lambda n: lambda f: lambda x: n(lambda g: lambda h: h(g(f)))(lambda u: x)(lambda u: u))(m)

check_numeral(_SUB(_THREE)(_ZERO), 3)
check_numeral(_SUB(_THREE)(_TWO), 1)
check_numeral(_SUB(_FIVE)(_TWO), 3)

## Logic and predicates

### Booleans

*   TRUE := λx.λy.x
*   FALSE := λx.λy.y

In [9]:
_TRUE = lambda x: lambda y: x
_FALSE = lambda x: lambda y: y

### Operators

*   AND := λp.λq.p q p
*   OR := λp.λq.p p q
*   NOT := λp.λa.λb.p b a
*   IFTHENELSE := λp.λa.λb.p a b

In [10]:
_AND = lambda p: lambda q: p(q)(p)
_OR = lambda p: lambda q: p(p)(q)
_NOT = lambda p: lambda a: lambda b: p(b)(a)
_IFTHENELSE = lambda p: lambda a: lambda b: p(a)(b)

assert _AND(_TRUE)(_TRUE)(1)(0) == 1
assert _AND(_FALSE)(_TRUE)(1)(0) == 0
assert _OR(_FALSE)(_TRUE)(1)(0) == 1
assert _NOT(_FALSE)(1)(0) == 1
assert _NOT(_TRUE)(1)(0) == 0
assert _IFTHENELSE(_TRUE)(42)(33) == 42
assert _IFTHENELSE(_FALSE)(42)(33) == 33

### Basic predicates

ISZERO := λn.n (λx.FALSE) TRUE

In [11]:
_ISZERO = lambda n: n(lambda x: (lambda x: lambda y: y))(lambda x: lambda y: x)

assert _ISZERO(_ZERO)(1)(0) == 1
assert _ISZERO(_TWO)(1)(0) == 0

LEQ := λm.λn.ISZERO (SUB m n)

In [12]:
_LEQ = lambda m: lambda n: (lambda n: n(lambda x: (lambda x: lambda y: y))(lambda x: lambda y: x))\
    ((lambda m: lambda n: n(lambda n: lambda f: lambda x: n(lambda g: lambda h: h(g(f)))(lambda u: x)(lambda u: u))(m))(m)(n))

    
assert _LEQ(_ZERO)(_ONE)(1)(0) == 1
assert _LEQ(_ONE)(_THREE)(1)(0) == 1
assert _LEQ(_THREE)(_ONE)(1)(0) == 0
assert _LEQ(_ONE)(_ONE)(1)(0) == 1

# Pairs

*   PAIR := λx.λy.λf.f x y
*   FIRST := λp.p TRUE
*   SECOND := λp.p FALSE
*   NIL := λx.TRUE
*   NULL := λp.p (λx.λy.FALSE)

In [13]:
_PAIR = lambda x: lambda y: lambda f: f(x)(y)
_FIRST = lambda p: p(lambda x: lambda y: x)
_SECOND = lambda p: p(lambda x: lambda y: y)
_NIL = lambda x: lambda x: lambda y: x
_NULL = lambda p: p(lambda x: lambda y: (lambda x: lambda y: y))

check_numeral(_FIRST(_PAIR(_ONE)(_TWO)), 1)
check_numeral(_SECOND(_PAIR(_ONE)(_TWO)), 2)

In [14]:
get_definition(_NULL)

u'lambda p: p(lambda x: lambda y: (lambda x: lambda y: y))'

In [22]:
import meta

ast = meta.decompiler.decompile_func(_NULL)
meta.asttools.print_ast(ast)

Lambda(args=arguments(args=[Name(ctx=Param(), 
                                 id='p')], 
                      defaults=[], 
                      kwarg=None, 
                      vararg=None), 
       body=Call(args=[Lambda(args=arguments(args=[Name(ctx=Param(), 
                                                        id='x')], 
                                             defaults=[], 
                                             kwarg=None, 
                                             vararg=None), 
                              body=Lambda(args=arguments(args=[Name(ctx=Param(), 
                                                                    id='y')], 
                                                         defaults=[], 
                                                         kwarg=None, 
                                                         vararg=None), 
                                          body=Lambda(args=arguments(args=[Name(ctx=Param(), 
                   